In [19]:
import pandas as pd
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
import re

In [13]:
proj_path = "/Users/gopal/Google Drive/_Research/Research projects/ML/manclassify/app_data/Thailand"
class_path = os.path.join(proj_path,"Thailand_classification")
ts_path = os.path.join(proj_path,"Thailand_download_timeseries")
# pd.read_csv("
os.listdir(class_path)

loc_id = 0

s2_csv_name = f"pt_ts_loc{loc_id}_s2.csv"
s2_csv_name



'pt_ts_loc0_s2.csv'

In [23]:
os.listdir(class_path)

['countif.xlsx', 'location_classification.csv']

In [210]:
class s2Dataset(Dataset):
    """Sentinel 2 dataset"""
    
    def __init__(self, proj_path, class_colname):
        """
        Args:
            proj_path (string): path to manclassify project
        """
        self.proj_path = proj_path
        proj_normpath = os.path.normpath(proj_path)
        proj_dirname = proj_normpath.split(os.sep)[-1]
        self.proj_name = re.sub("_classification$","",proj_dirname)
        self.class_path = os.path.join(proj_path, self.proj_name + "_classification")
        self.ts_path = os.path.join(proj_path, self.proj_name + "_download_timeseries")
        self.pt_classes = pd.read_csv(os.path.join(self.class_path,"location_classification.csv"))
        self.pt_classes = classes[['loc_id', class_colname]].dropna()
        # self.pt_classes['loc_id'] = self.pt_classes['loc_id'] + 10.5 # for testing index only
        self.classes = pd.unique(self.pt_classes[class_colname])
        self.labels = self.pt_classes.assign(val = 1).pivot_table(columns = class_colname, index = 'loc_id', values = 'val', fill_value= 0)

    
    def __getitem__(self, idx):
        date_range = pd.to_datetime(['2019-06-01','2020-05-31'])
        loc_id = self.labels.index[idx]
        s2_csv_name = f"pt_ts_loc{loc_id}_s2.csv"
        s2_csv_path = os.path.join(self.ts_path, s2_csv_name)
        s2_ts = pd.read_csv(s2_csv_path)
        
        # extract dates from image ids
        s2_ts['datestr'] = [re.sub("(^[0-9]+)[a-zA-Z].*","\\1",x) for x in s2_ts.image_id]
        s2_ts['date'] = pd.to_datetime(s2_ts.datestr, format = "%Y%m%d")
        s2_ts = s2_ts[(s2_ts.date >= date_range[0]) & (s2_ts.date <= date_range[1]) & (s2_ts.cloudmask == 0)]
        # s2_ts['date'] = [re.sub("(^[0-9]+)[a-zA-Z].*","\\1",x) for x in s2_ts.image_id]
        return s2_ts
        
    def __len__(self):
        return self.pt_classes.shape[0]

In [212]:
date_rangeX = pd.to_datetime(['2019-06-01','2020-05-31'])
s2_dataset = s2Dataset(proj_path = proj_path, class_colname = 'Subclass2019')
classes = pd.read_csv(os.path.join(s2_dataset.class_path,"location_classification.csv"))
# c2 = classes[['loc_id', 'Subclass2019']]#[!classes['Subclass2019'].isnan]
df = s2_dataset.__getitem__(4)
# df_datestr = re.sub("(^[0-9]+)[a-zA-Z].*","\\1",df['image_id'][0])#.index[0]#[0]#loc[1,1]#.iloc[1]
# pd.to_datetime(['2019-06-01','2020-05-31'])
df#[(df.date >= date_rangeX[0]) & (df.date <= date_rangeX[1])]
# (df.date >= date_rangeX[0]) & (df.date <= date_rangeX[1])

,B8,B4,B3,B2,clouds,cloudmask,shadows,probability,image_id,datestr,date
39,3904,235,471,236,0,0,0.0,1,20190621T033549_20190621T035429_T47NNJ,20190621,2019-06-21
43,2089,265,451,383,0,0,0.0,9,20190711T033549_20190711T034800_T47NPJ,20190711,2019-07-11
46,3745,280,538,329,0,0,0.0,2,20190820T033539_20190820T035122_T47NNJ,20190820,2019-08-20
47,3791,225,474,269,0,0,0.0,2,20190820T033539_20190820T035122_T47NPJ,20190820,2019-08-20
48,4465,392,712,562,0,0,0.0,29,20190929T033539_20190929T034838_T47NNJ,20190929,2019-09-29
49,3285,215,439,231,0,0,0.0,3,20191004T033601_20191004T035229_T47NNJ,20191004,2019-10-04
50,3276,179,404,187,0,0,0.0,4,20191004T033601_20191004T035229_T47NPJ,20191004,2019-10-04
51,3411,217,441,229,0,0,0.0,2,20191024T033821_20191024T034152_T47NNJ,20191024,2019-10-24
52,3380,155,358,141,0,0,0.0,1,20191024T033821_20191024T034152_T47NPJ,20191024,2019-10-24
53,2067,223,416,384,0,0,0.0,11,20191029T033849_20191029T035204_T47NNJ,20191029,2019-10-29


In [132]:
df['val'] = 1
labels_one_hot = df.pivot_table(columns = 'Subclass2019', index = 'loc_id', values = 'val', fill_value= 0)
labels_one_hot

Subclass2019,Crop(Double),Crop(Single),Forest,Golf,Mixed,Plantation,Unsure,Urban,Water
loc_id,,,,,,,,,
0,0,0,0,0,0,1,0,0,0
1,0,1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
496,0,1,0,0,0,0,0,0,0
497,0,1,0,0,0,0,0,0,0
498,0,0,0,0,0,1,0,0,0


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [29]:
proj_path

'/Users/gopal/Google Drive/_Research/Research projects/ML/manclassify/app_data/Thailand'